In [1]:
import sympy as sp

from symqups import s
from symqups.objects.base import PhaseSpaceObject, qpTypePSO, alphaTypePSO
from symqups.objects.scalars import q, p, alpha, alphaD, W, mu, _Primed, _DerivativeSymbol, hbar, _DePrimed, StateFunction
from symqups.objects.operators import qOp, pOp, annihilateOp, createOp, rho
from symqups.utils.algebra import define, qp2a, get_random_poly
from symqups.operations.quantization import s_quantize, naive_quantize
from symqups.objects import scalars
from symqups.objects.cache import _sub_cache
from symqups.operations.star_product import Bopp, Star, _replace_diff
from symqups.utils.multiprocessing import _mp_helper

a = alpha()
a_1 = alpha(1)
aop = annihilateOp()
ad = alphaD()
ad_1 = alphaD(1)
adop = createOp()
x = sp.Symbol("x")
qq = q()
pp = p()
s

s